In [17]:
# !pip install xgboost

In [18]:
import logging
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Any, Union, Optional, Tuple
import pickle
import json
import numpy as np
import pandas as pd
from scipy import stats
import copy
import sys

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score
)
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
class ModelPaths:
    """模型相關路徑管理"""
    BASE_DIR = Path("D:/Min/Python/Project/FA_Data")
    MODEL_DIR = BASE_DIR / "models"
    META_DIR = BASE_DIR / "meta_data"
    FEATURE_DIR = BASE_DIR / "features"
    RESULTS_DIR = BASE_DIR / "results"

    @classmethod
    def initialize(cls) -> None:
        """創建必要目錄結構"""
        directories = [cls.MODEL_DIR, cls.META_DIR, cls.FEATURE_DIR, cls.RESULTS_DIR]
        for directory in directories:
            directory.mkdir(parents=True, exist_ok=True)

    @classmethod
    def get_model_path(cls, stock_id: str, model_type: str) -> Path:
        """獲取模型儲存路徑"""
        return cls.MODEL_DIR / f"{stock_id}_{model_type}_model.pkl"
    
    @classmethod
    def get_result_path(cls, stock_id: str, model_type: str) -> Path:
        """獲取結果儲存路徑"""
        return cls.RESULTS_DIR / f"{stock_id}_{model_type}_results.json"

In [20]:
class BaseModel:
    """模型基礎類"""
    
    def __init__(self):
        """初始化"""
        self.model = None
        self.train_time = None
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def train(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        訓練模型
        
        Args:
            X: 特徵矩陣
            y: 標籤向量
        """
        try:
            self.logger.info(f"開始訓練模型，數據形狀: X={X.shape}, y={y.shape}")
            start_time = datetime.now()
            self.model.fit(X, y)
            self.train_time = (datetime.now() - start_time).total_seconds()
            self.logger.info(f"模型訓練完成，耗時: {self.train_time:.2f}秒")
        except Exception as e:
            self.logger.error(f"訓練過程出錯: {str(e)}")
            raise
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        預測標籤
        
        Args:
            X: 特徵矩陣
            
        Returns:
            預測的標籤值
        """
        try:
            if self.model is None:
                raise ValueError("模型尚未訓練")
            
            self.logger.info(f"開始預測，輸入數據形狀: {X.shape}")
            predictions = self.model.predict(X)
            self.logger.info(f"預測完成，輸出形狀: {predictions.shape}")
            return predictions
        except Exception as e:
            self.logger.error(f"預測過程出錯: {str(e)}")
            raise
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        預測概率
        
        Args:
            X: 特徵矩陣
            
        Returns:
            預測的概率值
        """
        try:
            if self.model is None:
                raise ValueError("模型尚未訓練")
                
            self.logger.info(f"開始預測概率，輸入數據形狀: {X.shape}")
            probabilities = self.model.predict_proba(X)
            self.logger.info(f"預測完成，輸出形狀: {probabilities.shape}")
            return probabilities
        except Exception as e:
            self.logger.error(f"預測概率過程出錯: {str(e)}")
            raise
        
    def save(self, path: Path) -> None:
        """保存模型"""
        try:
            path.parent.mkdir(parents=True, exist_ok=True)
            
            # 區分一般模型和集成模型
            if isinstance(self, EnsembleModel):
                save_data = {
                    'model_type': 'ensemble',
                    'sub_models': [
                        {
                            'model': model.model,
                            'train_time': model.train_time
                        }
                        for model in self.models
                    ],
                    'weights': self.weights,
                    'train_time': self.train_time
                }
            elif self.model is None:
                raise ValueError("沒有可保存的模型")
            else:
                save_data = {
                    'model_type': 'single',
                    'model': self.model,
                    'train_time': self.train_time
                }
                
            with open(path, 'wb') as f:
                pickle.dump(save_data, f)
                
        except Exception as e:
            self.logger.error(f"保存模型時出錯: {str(e)}")
            raise
            
    @classmethod
    def load(cls, path: Path) -> 'BaseModel':
        """載入模型"""
        if not path.exists():
            raise FileNotFoundError(f"找不到模型文件: {path}")
            
        with open(path, 'rb') as f:
            data = pickle.load(f)
            
        if data.get('model_type') == 'ensemble':
            # 重建子模型
            sub_models = []
            for sub_model_data in data['sub_models']:
                model = cls()
                model.model = sub_model_data['model']
                model.train_time = sub_model_data['train_time']
                sub_models.append(model)
                
            instance = EnsembleModel(
                models=sub_models,
                weights=data['weights']
            )
        else:
            instance = cls()
            instance.model = data['model']
            instance.train_time = data['train_time']
            
        return instance
    
    def get_feature_importance(self, feature_names: List[str]) -> Dict[str, float]:
        """
        獲取特徵重要性（如果模型支持）
        
        Args:
            feature_names: 特徵名稱列表
            
        Returns:
            特徵重要性字典
        """
        try:
            if self.model is None:
                raise ValueError("模型尚未訓練")
                
            if not hasattr(self.model, 'feature_importances_'):
                self.logger.warning("當前模型不支持特徵重要性")
                return {}
                
            self.logger.info("開始計算特徵重要性")
            importance_dict = dict(zip(feature_names, self.model.feature_importances_))
            self.logger.info("特徵重要性計算完成")
            return importance_dict
        except Exception as e:
            self.logger.error(f"獲取特徵重要性時出錯: {str(e)}")
            raise

In [21]:
class ModelFactory:
    """模型工廠類"""
    MODEL_MAPPINGS = {
        'random_forest': (RandomForestClassifier, {
            'n_estimators': 100,
            'max_depth': 5,
            'random_state': 42,
            'n_jobs': -1
        }),
        'xgboost': (xgb.XGBClassifier, {
            'max_depth': 6,
            'learning_rate': 0.1,
            'objective': 'binary:logistic',
            'random_state': 42
        })
    }
    
    @classmethod
    def create_model(cls, model_type: str, params: Optional[Dict] = None) -> BaseModel:
        """創建模型實例"""
        model_type = model_type.lower()
        if model_type not in cls.MODEL_MAPPINGS:
            raise ValueError(f"不支援的模型類型: {model_type}")
            
        model_class, default_params = cls.MODEL_MAPPINGS[model_type]
        final_params = {**default_params, **(params or {})}
        
        # 創建一個基礎模型實例
        model = BaseModel()
        model.model = model_class(**final_params)
        return model

In [22]:
class ModelEvaluator:
    """模型評估器"""
    
    @staticmethod
    def evaluate_classification(y_true: np.ndarray, y_pred: np.ndarray, 
                              y_prob: Optional[np.ndarray] = None) -> Dict[str, float]:
        """評估分類模型效果"""
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred)
        }
        
        if y_prob is not None:
            metrics['auc'] = roc_auc_score(y_true, y_prob[:, 1])
        return metrics
    
    @staticmethod
    def evaluate_feature_stability(X_train: np.ndarray, X_test: np.ndarray, 
                                 feature_names: List[str]) -> Dict[str, float]:
        """評估特徵穩定性"""
        eps = 1e-10
        stability_scores = {}
        for i, feature in enumerate(feature_names):
            train_dist = np.histogram(X_train[:, i], bins=20)[0] + eps
            test_dist = np.histogram(X_test[:, i], bins=20)[0] + eps
            psi = np.sum((train_dist - test_dist) * np.log(train_dist / test_dist))
            stability_scores[feature] = psi
        return stability_scores

    @staticmethod
    def evaluate_prediction_distribution(y_prob: np.ndarray) -> Dict[str, float]:
        """評估預測分布"""
        return {
            'prediction_mean': float(np.mean(y_prob[:, 1])),
            'prediction_std': float(np.std(y_prob[:, 1])),
            'prediction_skew': float(stats.skew(y_prob[:, 1])),
            'prediction_kurt': float(stats.kurtosis(y_prob[:, 1]))
        }

In [23]:
class ModelStabilityChecker:
    """檢查模型穩定性"""
    
    def check_model_stability(self, model: BaseModel, X: np.ndarray, y: np.ndarray, 
                            n_splits: int = 5) -> Dict[str, float]:
        """檢查模型穩定性"""
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        fold_scores = []
        prediction_variances = []
        
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            model_fold = copy.deepcopy(model)
            model_fold.train(X_train, y_train)
            y_pred_proba = model_fold.predict_proba(X_val)
            y_pred = (y_pred_proba[:, 1] >= 0.5).astype(int)
            
            fold_scores.append(accuracy_score(y_val, y_pred))
            prediction_variances.append(np.var(y_pred_proba[:, 1]))
        
        return {
            'score_mean': float(np.mean(fold_scores)),
            'score_std': float(np.std(fold_scores)),
            'score_cv': float(np.std(fold_scores) / np.mean(fold_scores)),
            'prediction_variance': float(np.mean(prediction_variances))
        }

In [24]:
class ModelOptimizer:
    """模型優化器"""
    
    def __init__(self, model_type: str, param_grid: Dict[str, List[Any]]):
        self.model_type = model_type
        self.param_grid = param_grid
        self.best_params = None
        self.best_score = None
        
    def optimize(self, X: np.ndarray, y: np.ndarray, 
                cv: int = 5, scoring: str = 'accuracy') -> Dict[str, Any]:
        """
        執行參數優化
        
        Args:
            X: 特徵矩陣
            y: 標籤向量
            cv: 交叉驗證折數
            scoring: 評分標準
            
        Returns:
            優化結果字典
        """
        logging.info(f"開始對 {self.model_type} 進行參數優化...")
        
        grid_search = GridSearchCV(
            ModelFactory.create_model(self.model_type).model,
            self.param_grid,
            cv=cv,
            scoring=scoring,
            n_jobs=-1,
            verbose=1
        )
        
        grid_search.fit(X, y)
        
        self.best_params = grid_search.best_params_
        self.best_score = grid_search.best_score_
        
        logging.info(f"優化完成. 最佳分數: {self.best_score:.4f}")
        logging.info(f"最佳參數: {self.best_params}")
        
        return {
            'best_params': self.best_params,
            'best_score': self.best_score,
            'cv_results': grid_search.cv_results_
        }

In [25]:
class DataValidator:
    """數據驗證類"""
    
    def __init__(self, stock_id: str, base_path: Path):
        self.stock_id = str(stock_id).zfill(4)  # 確保股票代碼格式正確
        self.base_path = base_path
        self.logger = logging.getLogger(self.__class__.__name__)
        
    def validate_and_load(self) -> pd.DataFrame:
        """驗證並載入數據"""
        try:
            # 檢查前置條件
            self._check_prerequisites()
            
            # 載入數據
            stock_data = self._load_data()
            
            # 驗證數據
            self._validate_data(stock_data)
            
            return stock_data
            
        except Exception as e:
            self.logger.error(f"數據驗證和載入失敗: {str(e)}")
            raise
            
    def _check_prerequisites(self) -> None:
        """檢查必要的前置數據文件是否存在"""
        # 檢查原始交易數據
        stock_data_file = self.base_path / "meta_data" / "stock_data_whole.csv"
        if not stock_data_file.exists():
            raise FileNotFoundError("找不到股票交易數據，請先執行 01_stock_data_collector.ipynb")
            
        # 檢查技術指標數據
        tech_indicator_file = self.base_path / "technical_analysis" / f"{self.stock_id}_indicators.csv"
        if not tech_indicator_file.exists():
            raise FileNotFoundError("找不到技術指標數據，請執行 02_technical_calculator.ipynb")
            
        # 檢查特徵數據
        feature_file = self.base_path / "meta_data" / "enhanced_features.csv"
        if not feature_file.exists():
            raise FileNotFoundError("找不到特徵數據，請執行 03_feature_generator.ipynb")
    
    def _load_data(self) -> pd.DataFrame:
        """載入數據"""
        feature_file = self.base_path / "meta_data" / "enhanced_features.csv"
        
        # 檢查文件大小
        self.logger.info(f"特徵文件大小: {feature_file.stat().st_size / (1024*1024):.2f} MB")
        
        # 讀取數據
        stock_data = pd.read_csv(feature_file, dtype={
            '證券代號': str,
            '日期': str
        }, low_memory=False)
        
        # 基本資訊記錄
        self.logger.info(f"總數據量: {len(stock_data)}")
        self.logger.info(f"唯一股票數: {stock_data['證券代號'].nunique()}")
        self.logger.info(f"日期範圍: {stock_data['日期'].min()} 到 {stock_data['日期'].max()}")
        
        # 篩選指定股票的數據
        stock_data = stock_data[stock_data['證券代號'] == self.stock_id].copy()
        self.logger.info(f"股票 {self.stock_id} 的數據量: {len(stock_data)}")
        
        return stock_data
    
    def _validate_data(self, df: pd.DataFrame) -> None:
        """驗證數據"""
        # 檢查數據量是否足夠
        if len(df) < 30:
            raise ValueError(f"股票 {self.stock_id} 的數據量不足，需要至少30天的數據")
            
        # 檢查必要特徵是否存在
        required_features = [
            '趨勢強度', '振幅', '量能趨勢', '量比', 'RSI', 'MACD',  # 基礎特徵
            '均線糾結度', '波動率', 'RSI_動能', 'MACD_動能',        # 技術特徵
            'KD_差值', '本益比_相對值', '技術綜合評分'             # 綜合指標
        ]
        
        missing_features = [f for f in required_features if f not in df.columns]
        if missing_features:
            raise ValueError(f"缺少以下特徵: {missing_features}")
            
        # 檢查並報告缺失值
        null_stats = df[required_features].isnull().sum()
        if null_stats.any():
            self.logger.warning("數據存在缺失值:")
            for feature, null_count in null_stats[null_stats > 0].items():
                self.logger.warning(f"{feature}: {null_count} 個缺失值")

class FeatureEngineer:
    """特徵工程類"""
    
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)
        
    def process(self, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, List[str]]:
        """處理特徵工程"""
        try:
            self.logger.info("開始特徵工程處理")
            
            # 清理列名（移除_x, _y後綴）
            df = self._clean_column_names(df)
            
            # 定義必要特徵
            required_features = [
                '趨勢強度', '振幅', '量能趨勢', '量比', 'RSI', 'MACD',  # 基礎特徵
                '均線糾結度', '波動率', 'RSI_動能', 'MACD_動能',        # 技術特徵
                'KD_差值', '本益比_相對值', '技術綜合評分'             # 綜合指標
            ]
            
            # 處理缺失值
            df = self._handle_missing_values(df, required_features)
            
            # 準備特徵和標籤
            X = df[required_features].values
            y = (df['收盤價'].shift(-1) > df['收盤價']).astype(int).values[:-1]
            X = X[:-1]  # 移除最後一行，以匹配標籤長度
            
            self.logger.info(f"特徵處理完成，特徵形狀: {X.shape}, 標籤形狀: {y.shape}")
            return X, y, required_features
            
        except Exception as e:
            self.logger.error(f"特徵工程處理失敗: {str(e)}")
            raise
    
    def _clean_column_names(self, df: pd.DataFrame) -> pd.DataFrame:
        """清理列名"""
        df = df.copy()
        columns_to_clean = [col for col in df.columns if col.endswith('_x') or col.endswith('_y')]
        for col in columns_to_clean:
            base_col = col[:-2]
            if f"{base_col}_x" in df.columns:
                df[base_col] = df[f"{base_col}_x"]
                df.drop([f"{base_col}_x", f"{base_col}_y"], axis=1, inplace=True, errors='ignore')
        return df
    
    def _handle_missing_values(self, df: pd.DataFrame, features: List[str]) -> pd.DataFrame:
        """處理缺失值"""
        df = df.copy()
        
        for feature in features:
            if feature not in df.columns:
                self.logger.warning(f"缺少特徵 {feature}，將被設為0")
                df[feature] = 0
            else:
                # 使用前向填充和後向填充的組合來處理缺失值
                df[feature] = df[feature].ffill().bfill()
                
                # 如果仍有缺失值，使用均值填充
                if df[feature].isnull().any():
                    feature_mean = df[feature].mean()
                    df[feature] = df[feature].fillna(feature_mean)
                    self.logger.info(f"特徵 {feature} 的剩餘缺失值使用均值 {feature_mean:.4f} 填充")
                    
        return df

In [26]:
class EnsembleModel(BaseModel):
    """混合模型"""
    def __init__(self, models: List[BaseModel], weights: Optional[List[float]] = None):
        super().__init__()
        self.models = models
        self.weights = weights or [1/len(models)] * len(models)
        self.model = self  # 添加這行,確保 self.model 不是 None
        
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """預測概率"""
        predictions = np.zeros((X.shape[0], 2))
        for model, weight in zip(self.models, self.weights):
            predictions += model.predict_proba(X) * weight
        return predictions

    def train(self, X: np.ndarray, y: np.ndarray) -> None:
        """訓練所有模型"""
        start_time = datetime.now()
        for model in self.models:
            model.train(X, y)
        self.train_time = (datetime.now() - start_time).total_seconds()
        
    def get_feature_importance(self, feature_names: List[str]) -> Dict[str, float]:
        """獲取集成模型的特徵重要性"""
        importances = {}
        for model, weight in zip(self.models, self.weights):
            model_importance = model.get_feature_importance(feature_names)
            for feature, importance in model_importance.items():
                importances[feature] = importances.get(feature, 0) + importance * weight
        return importances

In [27]:
class ModelTrainer:
    """模型訓練管理類"""
    
    def __init__(self, stock_id: str, base_path: Optional[Path] = None):
        self.stock_id = stock_id
        self.base_path = base_path or Path("D:/Min/Python/Project/FA_Data")
        self.logger = logging.getLogger(self.__class__.__name__)
        
        # 參數網格配置
        self.param_grids = {
            "random_forest": {
                'n_estimators': [300, 400, 500],
                'max_depth': [3, 4, 5, 6],
                'min_samples_split': [8, 10, 12],
                'min_samples_leaf': [2, 4],
                'max_features': ['sqrt', None],
                'class_weight': [None, 'balanced']
            },
            "xgboost": {
                'n_estimators': [100, 200, 300],
                'max_depth': [3, 4, 5, 6],
                'learning_rate': [0.01, 0.05, 0.1],
                'subsample': [0.8, 0.9, 1.0],
                'colsample_bytree': [0.8, 0.9, 1.0],
                'min_child_weight': [1, 3, 5]
            }
        }
        
    def run_batch_training(self, stock_ids: List[str], 
                         model_types: List[str] = ["random_forest", "xgboost"],
                         optimize: bool = False, 
                         cv_folds: int = 5,
                         batch_size: int = 1000) -> Dict[str, Any]:
        """執行批次股票訓練"""
        try:
            batch_results = {}
            
            # 1. 先收集所有股票的數據
            all_stock_data = {}
            for stock_id in stock_ids:
                validator = DataValidator(stock_id, self.base_path)
                stock_data = validator.validate_and_load()
                engineer = FeatureEngineer()
                X, y, feature_names = engineer.process(stock_data)
                all_stock_data[stock_id] = {
                    'X': X,
                    'y': y,
                    'feature_names': feature_names
                }
                
            # 2. 使用相似股票共同優化參數
            if optimize:
                # 使用所有股票的數據進行參數優化
                combined_X = np.vstack([data['X'] for data in all_stock_data.values()])
                combined_y = np.concatenate([data['y'] for data in all_stock_data.values()])
                
                optimized_params = {}
                for model_type in model_types:
                    optimizer = ModelOptimizer(model_type, self.param_grids[model_type])
                    opt_results = optimizer.optimize(combined_X, combined_y, cv=cv_folds)
                    optimized_params[model_type] = opt_results['best_params']
            
            # 3. 使用優化後的參數進行批次訓練
            for stock_id, data in all_stock_data.items():
                X_train, X_test, y_train, y_test = train_test_split(
                    data['X'], data['y'], test_size=0.2, shuffle=False
                )
                
                models = []
                for model_type in model_types:
                    params = optimized_params[model_type] if optimize else None
                    model = ModelFactory.create_model(model_type, params)
                    model.train(X_train, y_train)
                    models.append(model)
                
                # 創建並評估集成模型
                ensemble_model = EnsembleModel(models)
                results = self._evaluate_models(
                    ensemble_model, models, model_types,
                    {'X': X_train, 'y': y_train},
                    {'X': X_test, 'y': y_test},
                    data['feature_names'],
                    cv_folds
                )
                
                batch_results[stock_id] = results
                
                # 保存結果
                self._save_results(ensemble_model, results, data['feature_names'])
                
            # 產生批次處理總結報告
            summary = self._generate_batch_summary(batch_results)
            self._save_batch_results(batch_results, summary)
            
            return {
                'batch_results': batch_results,
                'summary': summary
            }
                
        except Exception as e:
            self.logger.error(f"批次處理執行失敗: {str(e)}")
            raise

    def _generate_batch_summary(self, batch_results: Dict) -> Dict[str, Any]:
        """生成批次處理總結報告"""
        try:
            summary = {
                'total_stocks': len(batch_results),
                'success_count': sum(1 for r in batch_results.values() if r is not None),
                'average_metrics': {
                    'accuracy': [],
                    'precision': [],
                    'recall': [],
                    'f1': [],
                    'auc': []
                },
                'feature_importance_summary': {},
                'processing_time': datetime.now().isoformat()
            }
            
            # 計算平均指標
            for results in batch_results.values():
                if results:
                    for metric in summary['average_metrics'].keys():
                        if metric in results:
                            summary['average_metrics'][metric].append(results[metric])
                    
                    # 累加特徵重要性
                    if 'feature_importance' in results:
                        for feature, importance in results['feature_importance'].items():
                            if feature not in summary['feature_importance_summary']:
                                summary['feature_importance_summary'][feature] = []
                            summary['feature_importance_summary'][feature].append(importance)
            
            # 計算平均值
            for metric in summary['average_metrics'].keys():
                values = summary['average_metrics'][metric]
                summary['average_metrics'][metric] = (
                    float(np.mean(values)) if values else None
                )
            
            # 計算特徵重要性平均值
            summary['feature_importance_summary'] = {
                feature: float(np.mean(values))
                for feature, values in summary['feature_importance_summary'].items()
            }
            
            return summary
            
        except Exception as e:
            self.logger.error(f"生成批次處理總結報告失敗: {str(e)}")
            raise

    def _save_batch_results(self, batch_results: Dict, 
                          summary: Dict[str, Any]) -> None:
        """儲存批次處理結果和總結報告"""
        try:
            # 儲存總結報告
            summary_path = ModelPaths.RESULTS_DIR / 'batch_training_summary.json'
            with open(summary_path, 'w', encoding='utf-8') as f:
                json.dump(summary, f, indent=4, ensure_ascii=False)
                
            self.logger.info(f"批次處理總結報告已儲存至: {summary_path}")
            
        except Exception as e:
            self.logger.error(f"儲存批次處理結果失敗: {str(e)}")
            raise
    
    def run_training_pipeline(self, model_types: List[str] = ["random_forest", "xgboost"],
                            optimize: bool = False, cv_folds: int = 5,
                            test_size: float = 0.2) -> Dict[str, Any]:
        """執行完整的訓練流程"""
        try:
            # 數據準備
            X, y, feature_names = self._prepare_data()
            train_data, test_data = self._split_data(X, y, test_size)
            
            # 訓練模型
            models = []
            for model_type in model_types:
                model = self._train_model(train_data, model_type, optimize, cv_folds)
                models.append(model)
            
            # 創建並評估集成模型
            ensemble_model = EnsembleModel(models)
            evaluation_results = self._evaluate_models(
                ensemble_model, models, model_types,
                train_data, test_data, feature_names, cv_folds
            )
            
            # 保存結果
            self._save_results(ensemble_model, evaluation_results, feature_names)
            return evaluation_results
            
        except Exception as e:
            self.logger.error(f"訓練流程執行失敗: {str(e)}")
            raise
    
    def _prepare_data(self) -> Tuple[np.ndarray, np.ndarray, List[str]]:
        """數據準備和特徵工程"""
        validator = DataValidator(self.stock_id, self.base_path)
        stock_data = validator.validate_and_load()
        
        engineer = FeatureEngineer()
        return engineer.process(stock_data)
    
    def _split_data(self, X: np.ndarray, y: np.ndarray, test_size: float) -> Tuple[Dict, Dict]:
        """數據分割"""
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, shuffle=False
        )
        return {
            'X': X_train, 
            'y': y_train
        }, {
            'X': X_test, 
            'y': y_test
        }

    def _train_model(self, train_data: Dict, model_type: str,
                    optimize: bool, cv_folds: int) -> BaseModel:
        """模型訓練"""
        model = ModelFactory.create_model(model_type)
        
        if optimize:
            optimizer = ModelOptimizer(model_type, self.param_grids[model_type])
            best_params = optimizer.optimize(
                train_data['X'],
                train_data['y'],
                cv=cv_folds
            )['best_params']
            model = ModelFactory.create_model(model_type, params=best_params)
            
        model.train(train_data['X'], train_data['y'])
        return model

    def _evaluate_model(self, model: BaseModel, test_data: Dict) -> Dict[str, Any]:
        """優化模型評估過程"""
        evaluator = ModelEvaluator()
        
        # 只預測一次
        y_prob = model.predict_proba(test_data['X'])
        y_pred = (y_prob[:, 1] >= 0.5).astype(int)
        
        return evaluator.evaluate_classification(
            test_data['y'],
            y_pred,
            y_prob
        )
    
    def _evaluate_models(self, ensemble_model: EnsembleModel, 
                        individual_models: List[BaseModel],
                        model_types: List[str], train_data: Dict,
                        test_data: Dict, feature_names: List[str],
                        cv_folds: int) -> Dict[str, Any]:
        """評估所有模型"""
        evaluator = ModelEvaluator()
        stability_checker = ModelStabilityChecker()
        
        results = {
            **self._evaluate_model(ensemble_model, test_data),
            'model_stability': stability_checker.check_model_stability(
                ensemble_model, train_data['X'], train_data['y'], cv_folds
            ),
            'feature_importance': ensemble_model.get_feature_importance(feature_names),
            'individual_model_metrics': {}
        }
        
        # 評估個別模型
        for model_type, model in zip(model_types, individual_models):
            results['individual_model_metrics'][model_type] = {
                'evaluation': self._evaluate_model(model, test_data),
                'stability': stability_checker.check_model_stability(
                    model, train_data['X'], train_data['y'], cv_folds
                )
            }
        
        return results
    
    def _save_results(self, model: BaseModel, results: Dict, feature_names: List[str]) -> None:
        """保存模型和結果
        
        Args:
            model: 要保存的模型
            results: 評估結果
            feature_names: 特徵名稱列表
        """
        try:
            # 獲取保存路徑
            model_path = ModelPaths.get_model_path(self.stock_id, model.__class__.__name__)
            result_path = ModelPaths.get_result_path(
                self.stock_id,
                model.__class__.__name__
            )

            # 更新結果
            results.update({
                'feature_importance': model.get_feature_importance(feature_names),
                'timestamp': datetime.now().isoformat()
            })

            # 確保目錄存在
            model_path.parent.mkdir(parents=True, exist_ok=True)
            result_path.parent.mkdir(parents=True, exist_ok=True)

            # 保存模型
            model.save(model_path)
            
            # 保存結果
            with open(result_path, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=4, ensure_ascii=False)

            self.logger.info(f"模型已保存到: {model_path}")
            self.logger.info(f"結果已保存到: {result_path}")
            
        except Exception as e:
            self.logger.error(f"保存結果時出錯: {str(e)}")
            # 嘗試清理可能的部分寫入
            try:
                if Path(model_path).exists():
                    Path(model_path).unlink()
                if Path(result_path).exists():
                    Path(result_path).unlink()
            except Exception as cleanup_error:
                self.logger.error(f"清理失敗的保存文件時出錯: {str(cleanup_error)}")
            raise

In [28]:
class StockScorer:
    def calculate_stock_score(self, model_results, historical_data):
        """計算股票綜合分數"""
        scores = {
            'prediction_score': self._calculate_prediction_score(model_results),
            'confidence_score': self._calculate_confidence_score(model_results),
            'accuracy_score': self._calculate_accuracy_score(model_results),
            'risk_score': self._calculate_risk_score(historical_data),
            'momentum_score': self._calculate_momentum_score(historical_data)
        }
        
        # 計算加權總分
        total_score = sum(score * weight for score, weight in zip(
            scores.values(), 
            [0.3, 0.2, 0.2, 0.15, 0.15]  # 權重可調整
        ))
        
        return total_score, scores

In [29]:
class BatchModelTrainer:
    def train_batch(self, stock_ids: List[str]):
        """優化的批次訓練"""
        # 分群訓練
        stock_groups = self._group_similar_stocks(stock_ids)
        
        results = {}
        for group in stock_groups:
            # 對每組使用相似的參數
            group_params = self._optimize_group_params(group)
            
            for stock_id in group:
                results[stock_id] = self._train_single_stock(
                    stock_id, 
                    base_params=group_params
                )
        
        return results

In [30]:
class RecommendationGenerator:
    def generate_recommendations(self, batch_results: Dict):
        """生成推薦清單"""
        stock_scores = []
        
        for stock_id, results in batch_results.items():
            score, details = self.scorer.calculate_stock_score(
                results,
                self.historical_data[stock_id]
            )
            
            stock_scores.append({
                'stock_id': stock_id,
                'total_score': score,
                'prediction_probability': results['prediction_proba'],
                'confidence': results['model_confidence'],
                'risk_level': details['risk_score'],
                'score_details': details
            })
        
        # 排序並產生推薦清單
        recommendations = sorted(
            stock_scores,
            key=lambda x: x['total_score'],
            reverse=True
        )
        
        return recommendations

In [31]:
def main(mode: str = 'single', stock_ids: Union[str, List[str]] = None, batch_size: int = 1000):
    try:
        # 設置日誌
        logging.basicConfig(
            level=logging.WARNING,  # 改為WARNING級別
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('model_training.log', encoding='utf-8'),  # 保存詳細日誌到文件
                logging.StreamHandler(sys.stdout)  # 控制台只顯示重要信息
            ]
        )
        # 文件處理器保持INFO級別以記錄詳細信息
        file_handler = logging.FileHandler('model_training.log', encoding='utf-8')
        file_handler.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        logging.getLogger().addHandler(file_handler)
        
        # 初始化路徑
        ModelPaths.initialize()
        
        # 如果是批次模式且未指定股票，則讀取所有股票
        if stock_ids is None and mode == 'batch':
            feature_file = ModelPaths.META_DIR / "enhanced_features.csv"
            all_stocks_df = pd.read_csv(feature_file, usecols=['證券代號'])
            stock_ids = all_stocks_df['證券代號'].unique().tolist()
            logging.info(f"處理所有股票，共 {len(stock_ids)} 支")
        
        # 單一股票模式
        if mode == 'single':
            stock_id = stock_ids if isinstance(stock_ids, str) else '2330'
            trainer = ModelTrainer(stock_id=stock_id)
            results = trainer.run_training_pipeline(
                model_types=["random_forest", "xgboost"],
                optimize=True,
                cv_folds=5
            )
            
            # 輸出詳細的單一股票分析結果
            print_single_stock_results(results)
            
        # 批次處理模式
        elif mode == 'batch':
            if isinstance(stock_ids, str):
                stock_ids = [stock_ids]
                
            # 建立批次處理群組
            stock_groups = [
                stock_ids[i:i + batch_size] 
                for i in range(0, len(stock_ids), batch_size)
            ]
            
            all_results = {}
            for group in stock_groups:
                logging.info(f"處理批次，股票數量: {len(group)}")
                
                # 對每個群組進行參數優化和訓練
                group_trainer = ModelTrainer(stock_id='')
                group_results = group_trainer.run_batch_training(
                    stock_ids=group,
                    model_types=["random_forest", "xgboost"],
                    optimize=True,
                    cv_folds=5
                )
                all_results.update(group_results['batch_results'])
            
            # 儲存訓練結果
            save_batch_results(all_results)
            
            # 輸出批次處理摘要
            print_batch_summary(all_results)
        
        logging.info("模型訓練完成")
        
    except Exception as e:
        logging.error(f"執行過程出錯: {str(e)}")
        raise

def print_single_stock_results(results: Dict[str, Any]) -> None:
    """打印單一股票的詳細分析結果"""
    print("\n模型評估結果:")
    print("-" * 40)
    
    # 模型性能指標
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    for metric in metrics:
        if metric in results:
            print(f"{metric:15}: {results[metric]:.4f}")
    
    # 模型穩定性
    if 'model_stability' in results:
        print("\n模型穩定性:")
        print("-" * 40)
        for metric, value in results['model_stability'].items():
            print(f"{metric:20}: {value:.4f}")
    
    # 特徵重要性
    if 'feature_importance' in results:
        print("\n特徵重要性:")
        print("-" * 40)
        sorted_features = sorted(
            results['feature_importance'].items(),
            key=lambda x: x[1],
            reverse=True
        )
        for feature, importance in sorted_features:
            print(f"{feature:20}: {importance:.4f}")

def print_batch_summary(results: Dict[str, Any]) -> None:
    """打印批次處理的摘要結果"""
    print("\n批次處理摘要:")
    print("-" * 40)
    
    # 計算總體統計
    total_stocks = len(results)
    successful_stocks = sum(1 for r in results.values() if r is not None)
    
    print(f"處理股票總數: {total_stocks}")
    print(f"成功處理數量: {successful_stocks}")
    
    # 計算平均性能指標
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    avg_metrics = {metric: [] for metric in metrics}
    
    for stock_results in results.values():
        if stock_results:
            for metric in metrics:
                if metric in stock_results:
                    avg_metrics[metric].append(stock_results[metric])
    
    print("\n平均性能指標:")
    for metric, values in avg_metrics.items():
        if values:
            avg_value = np.mean(values)
            print(f"{metric:15}: {avg_value:.4f}")

def save_batch_results(results: Dict[str, Any]) -> None:
    """儲存批次處理結果"""
    save_path = ModelPaths.RESULTS_DIR / f"batch_training_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    # 準備可序列化的結果
    serializable_results = {
        stock_id: {
            k: float(v) if isinstance(v, np.float32) else v
            for k, v in result.items()
            if k != 'feature_importance'  # 排除特徵重要性，另存為separate file
        }
        for stock_id, result in results.items()
        if result is not None
    }
    
    # 儲存結果
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(serializable_results, f, indent=4, ensure_ascii=False)
    
    logging.info(f"批次處理結果已儲存至: {save_path}")

In [32]:
main(mode='batch', stock_ids=['2330','3706','2317','2382','3231','2609','2354','6669','3661','3013'], batch_size=10)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Fitting 5 folds for each of 972 candidates, totalling 4860 fits

批次處理摘要:
----------------------------------------
處理股票總數: 10
成功處理數量: 10

平均性能指標:
accuracy       : 0.5153
precision      : 0.4993
recall         : 0.2581
f1             : 0.3207
auc            : 0.5100


In [33]:
# 1. 單一股票模式（指定股票）
# main(mode='single', stock_ids='2330')

# # 2. 批次處理模式（指定股票列表）
# main(mode='batch', stock_ids=['2330','3706','2317','2382','3231','2609','2354','6669','3661','3013'], batch_size=2)
# # 3. 批次處理模式（處理所有股票）
# main(mode='batch')  # 會自動處理所有股票